<div style="font-size: 200%; font-weight: bold; color: maroon;">GraphFrames</div>
<!-- Paulo Villegas, 2020 - v. 1.2 -->

In [ ]:
# Importamos el paquete
import graphframes as gf

In [ ]:
# Comprobemos que el paquete está cargado
spark.sparkContext.getConf().get('spark.jars.packages')

In [ ]:
# Físicamente ¿qué ficheros se han cargado?
modlist = spark.sparkContext.getConf().get('spark.jars')
for m in sorted((modlist or '').split(',')):
    print(m)

# Datos
Definimos los mismos datos que para el ejemplo de GraphX: la red de amigos

<img src='img/social_graph.png' width="500"/>

In [ ]:
nodes_rdd = sc.parallelize( [(1, "Alice", 28),
                         (2, "Bob", 27),
                         (3, "Charlie", 65),
                         (4, "David", 42),
                         (5, "Ed", 55),
                         (6, "Fran", 50)] )

In [ ]:
edges = sc.parallelize( 
  [(2, 1, 7),
   (2, 4, 2),
   (3, 2, 4),
   (3, 6, 3),
   (4, 1, 1),
   (5, 2, 2),
   (5, 3, 8),
   (5, 6, 3)]
)

Ahora convertimos los RDDs en DataFrames, teniendo cuidad de crear las columnas necesarias para GraphFrames: `id` (nodos) y `src`/`dst` (enlaces)

In [ ]:
from pyspark.sql import Row

Node = Row( 'id', 'name', 'age' )
node_rows = nodes_rdd.map( lambda x : Node(*x) ) 
node_df = spark.createDataFrame(node_rows)

In [ ]:
Edge = Row('src', 'dst', 'likes')
edge_rows = edges.map( lambda x : Edge(*x) )
edge_df = spark.createDataFrame(edge_rows)

In [ ]:
edge_df.printSchema()

# Creación
Creamos el grafo a partir de los DataFrames de nodos y enlaces

In [ ]:
g = gf.GraphFrame(node_df, edge_df)

In [ ]:
g

# Operaciones

## Grados

Obtenemos los grados de cada nodo:

In [ ]:
g.degrees.toPandas()

In [ ]:
g.degrees.join(node_df, 'id').toPandas()

In [ ]:
g.outDegrees.toPandas()

## Búsqueda de caminos

Buscamos el camino más corto entre dos nodos usando [BFS](https://graphframes.github.io/graphframes/docs/_site/api/scala/org/graphframes/lib/BFS.html)
* especificamos el primero por propiedad
* el segundo mediante una condición
* además imponemos una condición a los enlaces del camino

In [ ]:
# Specify edge filters or max path lengths.
ruta = g.bfs("name = 'Ed'", "(age > 40) AND (age < 50)",\
  edgeFilter="likes > 1", maxPathLength=3)

In [ ]:
ruta.toPandas()